In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import os
from datetime import datetime
from functools import reduce

# Importation de la base de données TRANSACTION_FACT


In [2]:
#precisser que MONTH_ID est un datetime au moment de l'importation
df_transac = pd.read_csv("DATA_ORIGINAL/TRANSACTION_FACT.csv",
                        parse_dates=['MONTH_ID'],
                        date_parser=lambda x : datetime.strptime(x, "%Y%m"))

In [3]:
# convertion de la variable MONTH_ID pour qu'elle soit au year-month level 
df_transac['MONTH_ID'] = df_transac['MONTH_ID'].dt.to_period('M')

In [4]:
df_transac.head()

,MEMBER_ID,RETAILER,MONTH_ID,TRANSACTIONS,AMOUNT_SPENT,BASE_POINTS_EARNED,BONUS_POINTS_EARNED,REWARD_POINTS_EARNED,CASH_BACK_POINTS_EARNED
0,7290,CATEGORY_3,2020-02,4,14.60,2,2,4.0,0.0
1,7290,CATEGORY_2,2020-04,3,106.15,19,0,19.0,0.0
2,7290,CATEGORY_4,2021-04,1,92.66,18,0,18.0,0.0
3,7290,CATEGORY_4,2021-12,1,365.32,73,0,73.0,0.0
4,7290,CATEGORY_3,2019-05,4,37.00,4,0,4.0,0.0


In [5]:
df_transac.shape

(45565311, 9)

In [6]:
# valeurs dupliquées
df_transac.duplicated().sum()

0

In [7]:
#sort values
df_transac.sort_values(['MEMBER_ID','MONTH_ID'], inplace=True)

# Analyse exploratoire de la Base de Donnees

In [8]:
# pourcentage de valeurs manquantes de chaque variable
df_transac.isnull().sum() / df_transac.shape[0] * 100

MEMBER_ID                   0.000000
RETAILER                   17.932008
MONTH_ID                    0.000000
TRANSACTIONS                0.000000
AMOUNT_SPENT                0.000000
BASE_POINTS_EARNED          0.000000
BONUS_POINTS_EARNED         0.000000
REWARD_POINTS_EARNED        0.168106
CASH_BACK_POINTS_EARNED     0.168106
dtype: float64

In [9]:
df_transac.dtypes

MEMBER_ID                      int64
RETAILER                      object
MONTH_ID                   period[M]
TRANSACTIONS                   int64
AMOUNT_SPENT                 float64
BASE_POINTS_EARNED             int64
BONUS_POINTS_EARNED            int64
REWARD_POINTS_EARNED         float64
CASH_BACK_POINTS_EARNED      float64
dtype: object

In [10]:
pd.set_option('float_format', '{:f}'.format)

In [11]:
# statistique descriptives variables numériques
df_transac.drop('MEMBER_ID', axis=1).describe()

,TRANSACTIONS,AMOUNT_SPENT,BASE_POINTS_EARNED,BONUS_POINTS_EARNED,REWARD_POINTS_EARNED,CASH_BACK_POINTS_EARNED
count,45565311.000000,45565311.000000,45565311.000000,45565311.000000,45488713.000000,45488713.000000
mean,4.049950,115.873998,21.371885,12.191385,22.788406,10.831380
std,11.036129,668.330765,133.415743,100.364028,145.704892,83.796463
min,0.000000,-351758.960000,-70353.000000,-31986.000000,-79991.000000,-18193.000000
25%,1.000000,12.230000,2.000000,0.000000,0.000000,0.000000
50%,2.000000,29.510000,5.000000,0.000000,1.000000,0.000000
75%,4.000000,76.590000,13.000000,2.000000,9.000000,5.000000
max,17915.000000,473815.830000,94762.000000,200069.000000,94762.000000,200069.000000


## <span style='background :yellow' > Variable MEMBER_ID </span>

In [12]:
# il y a difference de 3 entre table de membres et transanct, effacer ces 3 membres!
#nunique table membres =1202985
df_transac['MEMBER_ID'].nunique()

1202982

## <span style='background :yellow' > Variable RETAILER </span>

In [13]:
#cathegories
#probleme avec nan, c'est la cathegories "autres"
df_transac.RETAILER.unique()

array(['CATEGORY_1', 'CATEGORY_3', 'CATEGORY_2', nan, 'CATEGORY_5',
       'CATEGORY_4'], dtype=object)

In [14]:
# Pourcentage de transaction dans chaque catégorie
df_transac.RETAILER.value_counts() / df_transac.shape[0] * 100

CATEGORY_2   36.545702
CATEGORY_3   17.484176
CATEGORY_1   11.433566
CATEGORY_4   10.060785
CATEGORY_5    6.543763
Name: RETAILER, dtype: float64

In [15]:
#nan vont etre asignées à catégorie "autres" = CATEGORY_6
# Remplacer les NaN par CATEGORY_6  
remplacement = {"RETAILER": 'CATEGORY_6'}
df_transac =df_transac.fillna(remplacement)

df_transac

,MEMBER_ID,RETAILER,MONTH_ID,TRANSACTIONS,AMOUNT_SPENT,BASE_POINTS_EARNED,BONUS_POINTS_EARNED,REWARD_POINTS_EARNED,CASH_BACK_POINTS_EARNED
1809896,44,CATEGORY_1,2019-06,2,11.090000,1,0,1.000000,0.000000
1809895,44,CATEGORY_1,2019-12,3,21.610000,2,0,2.000000,0.000000
1809893,44,CATEGORY_1,2020-01,1,7.530000,1,0,1.000000,0.000000
1809894,44,CATEGORY_1,2020-02,3,19.060000,2,0,2.000000,0.000000
33674028,112,CATEGORY_3,2019-01,1,9.840000,1,0,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...
29176466,1019023002,CATEGORY_4,2021-08,3,477.470000,95,30,125.000000,0.000000
29176477,1019023002,CATEGORY_4,2021-09,1,349.340000,69,0,69.000000,0.000000
29176460,1019023002,CATEGORY_4,2021-10,1,444.050000,88,0,88.000000,0.000000
29176475,1019023002,CATEGORY_4,2021-11,1,604.590000,120,0,120.000000,0.000000


In [16]:
df_transac.RETAILER.unique()

array(['CATEGORY_1', 'CATEGORY_3', 'CATEGORY_2', 'CATEGORY_6',
       'CATEGORY_5', 'CATEGORY_4'], dtype=object)

In [17]:
# calculer a nouveau le Pourcentage de transaction dans chaque catégorie
df_transac.RETAILER.value_counts() / df_transac.shape[0] * 100

CATEGORY_2   36.545702
CATEGORY_6   17.932008
CATEGORY_3   17.484176
CATEGORY_1   11.433566
CATEGORY_4   10.060785
CATEGORY_5    6.543763
Name: RETAILER, dtype: float64

## <span style='background :yellow' > Variable AMOUNT_SPENT </span>

In [18]:
# des obs avec des AMOUNT SPENT inférieures à 0
nb_obs_inf_zero_as= df_transac[df_transac.AMOUNT_SPENT < 0].shape[0]
pr_obs_inf_zero_as= df_transac[df_transac.AMOUNT_SPENT < 0].shape[0] / df_transac.shape[0] * 100

df_test =df_transac.groupby(['MEMBER_ID'], as_index=False)['AMOUNT_SPENT'].sum()
df_test_2 =df_test[df_test.AMOUNT_SPENT == 0].shape[0] 

print('nb des obs avec des AMOUNT SPENT inférieures à 0: ',nb_obs_inf_zero_as)
print('pourcentage des obs avec des AMOUNT SPENT inférieures à 0: ',pr_obs_inf_zero_as)
print('clients qui ont des montants dépensés égaux à 0 :', df_test_2)

nb des obs avec des AMOUNT SPENT inférieures à 0:  118549
pourcentage des obs avec des AMOUNT SPENT inférieures à 0:  0.26017379756279946
clients qui ont des montants dépensés égaux à 0 : 26521


In [19]:
df_transac.drop('AMOUNT_SPENT', axis=1, inplace=True)

## <span style='background :yellow' > Variable BASE_POINTS_EARNED </span>

In [20]:
# des obs avec des BASE_POINTS_EARNED inférieures à 0
nb_obs_inf_zero_bp= df_transac[df_transac.BASE_POINTS_EARNED < 0].shape[0]
pr_obs_inf_zero_bp= df_transac[df_transac.BASE_POINTS_EARNED < 0].shape[0] / df_transac.shape[0] * 100

print('nb des obs avec des BASE_POINTS_EARNED inférieures à 0: ',nb_obs_inf_zero_bp)
print('pourcentage des obs BASE_POINTS_EARNED inférieures à 0: ',pr_obs_inf_zero_bp)

nb des obs avec des BASE_POINTS_EARNED inférieures à 0:  123796
pourcentage des obs BASE_POINTS_EARNED inférieures à 0:  0.27168913650123006


## <span style='background :yellow' > Variable BONUS_POINTS_EARNED </span>

In [21]:
# des obs avec des BASE_POINTS_EARNED inférieures à 0
nb_obs_inf_zero_bonusp= df_transac[df_transac.BONUS_POINTS_EARNED < 0].shape[0]
pr_obs_inf_zero_bonusp= df_transac[df_transac.BONUS_POINTS_EARNED < 0].shape[0] / df_transac.shape[0] * 100

print('nb des obs avec des BONUS_POINTS_EARNED inférieures à 0: ',nb_obs_inf_zero_bonusp)
print('pourcentage des obs BONUS_POINTS_EARNED inférieures à 0: ',pr_obs_inf_zero_bonusp)

nb des obs avec des BONUS_POINTS_EARNED inférieures à 0:  9050
pourcentage des obs BONUS_POINTS_EARNED inférieures à 0:  0.019861600417914408


## <span style='background :yellow' > Variable REWARD_POINTS_EARNED </span>

In [22]:
# des obs avec des BASE_POINTS_EARNED inférieures à 0
nb_obs_inf_zero_rp= df_transac[df_transac.REWARD_POINTS_EARNED < 0].shape[0]
pr_obs_inf_zero_rp= df_transac[df_transac.REWARD_POINTS_EARNED < 0].shape[0] / df_transac.shape[0] * 100

print('nb des obs avec des REWARD_POINTS_EARNED inférieures à 0: ',nb_obs_inf_zero_rp)
print('pourcentage des obs REWARD_POINTS_EARNED inférieures à 0: ',pr_obs_inf_zero_rp)

nb des obs avec des REWARD_POINTS_EARNED inférieures à 0:  84646
pourcentage des obs REWARD_POINTS_EARNED inférieures à 0:  0.1857685114889263


## <span style='background :yellow' > Variable CASH_BACK_POINTS_EARNED </span>

In [23]:
# des obs avec des BASE_POINTS_EARNED inférieures à 0
nb_obs_inf_zero_cbp= df_transac[df_transac.CASH_BACK_POINTS_EARNED < 0].shape[0]
pr_obs_inf_zero_cbp= df_transac[df_transac.CASH_BACK_POINTS_EARNED < 0].shape[0] / df_transac.shape[0] * 100

print('nb des obs avec des CASH_BACK_POINTS_EARNED inférieures à 0: ',nb_obs_inf_zero_cbp)
print('pourcentage des obs CASH_BACK_POINTS_EARNED inférieures à 0: ',pr_obs_inf_zero_cbp)

nb des obs avec des CASH_BACK_POINTS_EARNED inférieures à 0:  44330
pourcentage des obs CASH_BACK_POINTS_EARNED inférieures à 0:  0.0972889222680824


#  Creation de Variables 

## <span style='background :yellow' > Variable COVID_TRANSACTIONS_PCT_CHANGE </span>

Croissance / décroissance transaction covid

In [24]:
df_pre_covid = df_transac[(df_transac['MONTH_ID'] >= '2019-03') & (df_transac['MONTH_ID'] < '2020-03')].copy()

In [25]:
df_pre_covid = df_pre_covid.groupby('MEMBER_ID', as_index=False)['TRANSACTIONS'].sum()

In [26]:
df_pre_covid.rename(columns={'TRANSACTIONS':'TRANSACTIONS_PRE_COVID'}, inplace=True)

In [27]:
df_covid = df_transac[(df_transac['MONTH_ID'] >= '2020-03') & (df_transac['MONTH_ID'] < '2021-02')].copy()

In [28]:
df_covid = df_covid.groupby('MEMBER_ID', as_index=False)['TRANSACTIONS'].sum()

In [29]:
df_covid.rename(columns={'TRANSACTIONS':'TRANSACTIONS_COVID'}, inplace=True)

In [30]:
df_merged = pd.merge(df_pre_covid, df_covid, on='MEMBER_ID', how='inner')

In [31]:
df_merged['COVID_TRANSACTIONS_PCT_CHANGE'] = df_merged[['TRANSACTIONS_PRE_COVID', 'TRANSACTIONS_COVID']].pct_change(axis=1)['TRANSACTIONS_COVID']

In [32]:
df_merged.describe()

,MEMBER_ID,TRANSACTIONS_PRE_COVID,TRANSACTIONS_COVID,COVID_TRANSACTIONS_PCT_CHANGE
count,942373.000000,942373.000000,942373.000000,942373.000000
mean,420762406.524739,78.977860,57.115437,0.169129
std,492094722.793499,166.705537,123.013424,5.358488
min,112.000000,1.000000,1.000000,-0.999755
25%,7216420.000000,15.000000,11.000000,-0.513761
50%,15909231.000000,41.000000,30.000000,-0.250000
75%,1007705838.000000,94.000000,72.000000,0.073171
max,1019023002.000000,48827.000000,64775.000000,2815.304348


In [ ]:
df_merged

In [33]:
df_pct_change_covid = df_merged.loc[:, ['MEMBER_ID', 'COVID_TRANSACTIONS_PCT_CHANGE']]

## <span style='background :yellow' > Variable Recency </span>

Nombre de jours écoulés depuis la transaction la plus récente <br>
On utilise la dernière date d'achat pour tous les membres



In [34]:
df_recency = df_transac.groupby('MEMBER_ID')['MONTH_ID'].last().reset_index(name='LAST_DATE_TRANSACTION')

In [35]:
df_recency

,MEMBER_ID,LAST_DATE_TRANSACTION
0,44,2020-02
1,112,2021-09
2,123,2020-10
3,170,2021-12
4,183,2021-12
...,...,...
1202977,1019008669,2021-12
1202978,1019011271,2021-12
1202979,1019016174,2021-12
1202980,1019019110,2021-12


In [36]:
snapshot_date = df_transac.MONTH_ID.max()
snapshot_date

Period('2021-12', 'M')

In [37]:
df_recency['RECENCY'] = df_recency.LAST_DATE_TRANSACTION.apply(lambda x: (snapshot_date - x).n * 30)

In [38]:
df_recency

,MEMBER_ID,LAST_DATE_TRANSACTION,RECENCY
0,44,2020-02,660
1,112,2021-09,90
2,123,2020-10,420
3,170,2021-12,0
4,183,2021-12,0
...,...,...,...
1202977,1019008669,2021-12,0
1202978,1019011271,2021-12,0
1202979,1019016174,2021-12,0
1202980,1019019110,2021-12,0


In [39]:
df_recency.drop('LAST_DATE_TRANSACTION', axis=1, inplace=True)

In [40]:
df_recency.shape

(1202982, 2)

## <span style='background :yellow' > Variable Mono catégorie / poly catégorie </span>




In [41]:
df_number_categories = df_transac.groupby('MEMBER_ID')['RETAILER'].nunique().reset_index(name='NUMBER_CATEGORIES_PURCHASED_FROM')

In [42]:
df_number_categories

,MEMBER_ID,NUMBER_CATEGORIES_PURCHASED_FROM
0,44,1
1,112,4
2,123,3
3,170,5
4,183,5
...,...,...
1202977,1019008669,4
1202978,1019011271,4
1202979,1019016174,3
1202980,1019019110,5


In [43]:
df_number_categories.shape

(1202982, 2)

# agregation df_reward

In [44]:
df_transac_final = df_transac.groupby('MEMBER_ID')[['BASE_POINTS_EARNED', 'BONUS_POINTS_EARNED', 'REWARD_POINTS_EARNED', 'CASH_BACK_POINTS_EARNED']].sum().reset_index()

In [45]:
df_transac_final

,MEMBER_ID,BASE_POINTS_EARNED,BONUS_POINTS_EARNED,REWARD_POINTS_EARNED,CASH_BACK_POINTS_EARNED
0,44,6,0,6.000000,0.000000
1,112,163,87,250.000000,0.000000
2,123,41,25,66.000000,0.000000
3,170,1657,332,1989.000000,0.000000
4,183,429,680,1109.000000,0.000000
...,...,...,...,...,...
1202977,1019008669,5221,816,6037.000000,0.000000
1202978,1019011271,696,348,0.000000,1044.000000
1202979,1019016174,560,646,0.000000,1206.000000
1202980,1019019110,363,156,0.000000,519.000000


In [46]:
df_transac_final.shape

(1202982, 5)

# Jointure

In [47]:
df_to_be_merged = [df_transac_final, df_recency, df_number_categories, df_pct_change_covid]

In [48]:
df_transac_merged = reduce(lambda left,right: pd.merge(left,right,on=['MEMBER_ID'],
                                            how='left'), df_to_be_merged)

In [49]:
df_transac_merged

,MEMBER_ID,BASE_POINTS_EARNED,BONUS_POINTS_EARNED,REWARD_POINTS_EARNED,CASH_BACK_POINTS_EARNED,RECENCY,NUMBER_CATEGORIES_PURCHASED_FROM,COVID_TRANSACTIONS_PCT_CHANGE
0,44,6,0,6.000000,0.000000,660,1,NaN
1,112,163,87,250.000000,0.000000,90,4,-0.392857
2,123,41,25,66.000000,0.000000,420,3,0.500000
3,170,1657,332,1989.000000,0.000000,0,5,0.544444
4,183,429,680,1109.000000,0.000000,0,5,0.061856
...,...,...,...,...,...,...,...,...
1202977,1019008669,5221,816,6037.000000,0.000000,0,4,-0.441176
1202978,1019011271,696,348,0.000000,1044.000000,0,4,-0.047244
1202979,1019016174,560,646,0.000000,1206.000000,0,3,-0.552632
1202980,1019019110,363,156,0.000000,519.000000,0,5,-0.156522


In [50]:
df_transac_merged.shape

(1202982, 8)

In [51]:
output_to_csv = True

In [52]:
if output_to_csv:
    df_transac_merged.to_csv('DATA_PRETRAITEMENT/df_transaction.csv', index=False)